In [13]:
%load_ext autoreload
%autoreload 2

import numpy as np
from generateGif import create_animated_png,create_animated_gif
from geom import find_point_C

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
class SimulationParams:
    DA_x:float
    DA_y:float
    AB_length:float
    DC_length:float
    BC_length:float

In [17]:
def sim(params: SimulationParams):
    """
    Делает симуляцию
    Параметры:
    offset (tuple или list): Вектор смещения центра вращения (x0, y0).
    length (float): Длина отрезка.

    Возвращает:
    X (numpy.ndarray): Матрица координат X размером (360, 6).
    Y (numpy.ndarray): Матрица координат Y размером (360, 6).
    Порядок точек по столбцам: A, B, C, D, E, F
    """
    A_x = params.DA_x
    A_y = params.DA_y
    AB_length = params.AB_length
    углы = np.deg2rad(np.arange(0, 360))  # Углы в радианах от 0 до 359 градусов

    x_A = np.full(360, A_x)
    y_A = np.full(360, A_y)

    x_B = A_x + AB_length * np.cos(углы)
    y_B = A_y + AB_length * np.sin(углы)

    x_C=np.zeros((360))
    y_C=np.zeros((360))

    # Создаем матрицы с повторяющимися координатами центра и вычисленными концами
    X = np.column_stack((x_A, x_B))
    Y = np.column_stack((y_A, y_B))

    return X, Y

In [18]:
params = SimulationParams()
params.AB_length = 1
params.DC_length = 3
params.BC_length = 4
params.DA_x = 4
params.DA_y = 1

X, Y = sim(params)
create_animated_gif(X, Y, [0], [1], "tests.gif", 3000)

Анимированный GIF сохранен как tests.gif.
